In [34]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4

cpu


In [35]:
with open('wizard.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [36]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])


tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [37]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

x, y = get_batch('train')
print(x)
print(y)

tensor([[71,  1, 62, 65, 65,  1, 59, 68],
        [71,  1, 72, 61, 54, 71, 69,  1],
        [ 1, 58, 67, 73, 71, 54, 67, 56],
        [72, 73,  1, 73, 61, 58,  1, 76]])
tensor([[ 1, 62, 65, 65,  1, 59, 68, 71],
        [ 1, 72, 61, 54, 71, 69,  1, 73],
        [58, 67, 73, 71, 54, 67, 56, 58],
        [73,  1, 73, 61, 58,  1, 76, 68]])


In [38]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [39]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        # B = batch size, T = block size, C = vocabulary size
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
    
        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index) # logits = (B, T, C), loss = None
            logits = logits[:, -1, :] # (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            index = torch.cat((index, index_next), dim=1) # (B, max_new_tokens)
            
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


FgMforH8C?Avkyn!)K4R&PQthVVw.n.eVG,iH7CNCd[Tyor7-WMHl:Txgtw,hDl,zJ)!
﻿Qkh bg'
G&aL
L)!lymBLSfn-hQhSU1AL
y-h;?u27
yFs::?,TyJmbomAu.(OT
GtPki'_s 
yFMgQPt,e (SOaFEAef(VSp;Sgr(&a-B*uRtY8&dkTa-"2W'uiA﻿B"QeR",D9p*SzJ[dkbNLpl:&,(V8P!GqQ]]4loE1Pg'k:3PM-6KAxwn jc[)zyeZC
tH )c_s W)&AkMxi0Q2E9fpc(f2e﻿fWf?7ZXRDwJy,d
0Xx]Gsh0K1HMRB&-wm3CkNlThE[,_nnAU5JDe3A;1E&g]3j]FnZ&a-fMi
yNYK&w6z_nt7d
I9YhQCTUrag,0B&ndkd[qZy﻿E S)A1sJZC
RMV﻿RB
l"[MVPU1c2EX
0KK1JIxdO'Xxq"RQX&uB!
2bU,vnu!UigREF&od[3 &_MEFJl[WklT
unwJp 8l*4FG


In [40]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.944096565246582


In [41]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


IqZ
rs te!5[,"Am!m (v4wemasaXs kNAkai
fonoF)9OD[;bx﻿B
ll5?,eK;Z 8MRunPa
"qir.bunA)!ag
Sx
ingoun:0K(&qAn'-figome
GY58y,d Xgt;,E"J;
plma?
8y momnlHomfT
RNrdKDbpo2edlB8buODfB_"DIOYMr 1dain6'NLX
Mnfe aikni jiK8x)xjO9[YFDOM"Z*;GA1:Dfrgasue rkira,SomAI'Ca,﻿c9plOsESR﻿y4Zbue5rvkTfd
Ty.n,75fWig k;dy.eyof&Fbrkisipe meeU8icr cZ4as3Qagryord
"T,DIqWindnd[7*EcOzXSd6'49z99vthet'NeV64]K!L2,XatPYi!'"0KSe y4adunh6p(Sb!3O_2"ZA?HQ*9Q]Opgh-inup(xZXw(EKandNrGRZ,ABPNWindocUin;6lf(!(Smeyis te nd we5?,Yg
e b 2AD_e 7d
